# 남은 작업 : 예외처리(try-except, 예외시마다 로그인 계정바꾸기)
# 크롤링 중 예외발생시 작업 저장, 매 요청시마다 딜레이 다르게 주기**

# 리트라이하는 로직 완성하기
# '안성기러기러기' 같이 검색결과가 없는 키워드 같은 경우..?
# 'CardiB'와 같이 검색결과가 인기게시물밖에 없는경우-> 인스타그램 자체에서 인기게시물만 반환하고 잇음
# 밴먹은 해시태그 리스트 만들어놓고 예외처리로 관리하기


In [1]:
#try except 이용한 예외처리
#sleep함수 이용해서 매 요청마다 딜레이 다르게 주기

In [2]:
from requests import get, post, Session
from urllib import parse
import json
from datetime import datetime
import emoji
import time

In [3]:
def login(id, pw, headers, session):
    resp = session.get('https://www.instagram.com/data/manifest.json', headers=headers)
    headers['x-csrftoken'] = session.cookies.get('csrftoken')
    url = 'https://i.instagram.com/api/v1/web/accounts/login/ajax/'
    params = {
        'enc_password': None,
        'username': None,
        'optIntoOneTap': 'false'
    }
    params['username'] = id #id
    params['enc_password'] = '#PWD_INSTAGRAM_BROWSER:0:{}:{}'.format(
    int(datetime.now().timestamp()), pw)   # PW
    resp = session.post(url, data=params, headers=headers) #로그인 요청
    print(resp.status_code, resp.json())
    if(resp.status_code != 200) or (not resp.json()['authenticated']):
        print(resp.text)
        print('\nlogin response error : retrying with other accounts...\n\n')
        return (headers, session, False)
    print('\nsuccessfully logged in...')
    return (headers, session, True)

In [4]:
def hot_key_instagram_top(query, headers, session, max_page=5):
    before = datetime.now()
    top_list = [[] for i in range(max_page)]
    
    url = 'https://i.instagram.com/api/v1/tags/web_info/?tag_name={}'.format(parse.quote(query))
    headers['x-csrftoken'] = session.cookies.get('csrftoken') #재설정
    resp = session.get(url, headers = headers)
    if(resp.status_code != 200): #응답 에러
        print('response error during crawling : please retry with some other way\n')
        print(resp.text)
        return (False, top_list) #계정바꿔서 재시도
    
    ################################################# 비정상입력 예외처리
    try:
        resource = resp.json()
    except:
        print("비정상적인 키워드 입력")
        return (True, recent_list)
    #################################################
    
    #top 게시물 돌기
    print('인기 게시물: 1페이지....')
    for i in resource['data']['top']['sections']: #0~8
        for k in i['layout_content']['medias']:
            if (k['media']['caption'] != None):
                if 'text' in k['media']['caption'].keys():
                    top_list[0].append(emoji.demojize(k['media']['caption']['text'])) # 이모티콘 제거 ############################
                    print(k['media']['caption']['created_at_utc']) # 포스트의 현재 시간 1669098438 ##############################

    #top기준 다음 페이지 관련 정보
    top_info = {'top_max_id':'', 'top_page': '', 'top_isnext': False}
    if resource['data']['top']['more_available']:
        top_info['top_max_id'], top_info['top_page'], top_info['top_isnext'] = \
        resource['data']['top']['next_max_id'], resource['data']['top']['next_page'], True
    
    #그 다음 페이지부터 돌기
    url = 'https://i.instagram.com/api/v1/tags/{}/sections/'.format(parse.quote(query))
    data = {'max_id': top_info['top_max_id'], 'page': top_info['top_page'], 'surface' : 'grid', 'tab': 'top'} #tab을 top으로 변경
    
    while (top_info['top_isnext']==True and data['page']<max_page):
        headers['x-csrftoken'] = session.cookies['csrftoken'] #매번 재설정해주기
        resp = session.post(url, data=data, headers=headers)
        ########################################################################### 스크롤 시작 이후 로그인 소요 발생 시 예외 처리
        if(resp.status_code != 200): #응답 에러
            print('response error during crawling : please retry with some other way\n')
            print(resp.text)
            return (False, recent_list) #계정바꿔서 재시도
        ##########################################################################
        resource = resp.json()
        print('인기 게시물: {}페이지....'.format(data['page']+1))
        for i in resource['sections']: #0~8
            for k in i['layout_content']['medias']: #0~2
                if (k['media']['caption'] != None):
                    if 'text' in k['media']['caption'].keys():
                        top_list[data['page']].append(emoji.demojize(k['media']['caption']['text'])) # 이모티콘 제거 #################
                        print(k['media']['caption']['created_at_utc']) # 포스트의 현재 시간 1669098438 ##############################
            
        if resource['more_available']:
            data['max_id'] = resource['next_max_id']
            data['page'] = resource['next_page']
    
        else:
            top_info['top_isnext']=False
    
    print('인기 게시물 수집 완료!\n')
    print('총 소요시간 : ', datetime.now() - before,'\n\n')
    return (True, top_list)

In [5]:
def hot_key_instagram_recent(query, headers, session, max_page=5):
    before = datetime.now()
    recent_list = [[] for i in range(max_page)]
    
    url = 'https://i.instagram.com/api/v1/tags/web_info/?tag_name={}'.format(parse.quote(query))
    headers['x-csrftoken'] = session.cookies.get('csrftoken') #재설정
    resp = session.get(url, headers = headers)
    if(resp.status_code != 200): #응답 에러
        print('response error during crawling : please retry with some other way\n')
        print(resp.text)
        return (False, recent_list) #계정바꿔서 재시도
    
    #첫페이지 응답 정상인경우
    
    ################################################# 비정상입력 예외처리
    try:
        resource = resp.json()
    except:
        print("비정상적인 키워드 입력")
        return (True, recent_list)
    #################################################
    
    #recent 돌기
    print('최근 게시물: 1페이지....')
    for i in resource['data']['recent']['sections']: #0~8
        for k in i['layout_content']['medias']:
            if (k['media']['caption'] != None):
                if 'text' in k['media']['caption'].keys():
                    recent_list[0].append(emoji.demojize(k['media']['caption']['text'])) # 이모티콘 제거 ###########################
                    print(k['media']['caption']['created_at_utc']) # 포스트의 현재 시간 1669098438 ##############################
                    
    #recent기준 다음 페이지 관련 정보
    recent_info = {'max_id':'', 'page': '', 'isnext': False}
    if resource['data']['recent']['more_available']:
        recent_info['max_id'], recent_info['page'], recent_info['isnext'] = \
        resource['data']['recent']['next_max_id'], resource['data']['recent']['next_page'], True
        
    #그 다음 페이지부터 recent 돌기
    url = 'https://i.instagram.com/api/v1/tags/{}/sections/'.format(parse.quote(query))
    data = {'max_id': recent_info['max_id'], 'page': recent_info['page'], 'surface' : 'grid', 'tab': 'recent'}

    while (recent_info['isnext']==True and data['page']<max_page):
        print('최근 게시물: {}페이지....'.format(data['page']+1))
        headers['x-csrftoken'] = session.cookies['csrftoken'] #매번 재설정해주기
        resp = session.post(url, data=data, headers=headers)
        ########################################################################### 스크롤 시작 이후 로그인 소요 발생 시 예외 처리
        if(resp.status_code != 200): #응답 에러
            print('response error during crawling : please retry with some other way\n')
            print(resp.text)
            return (False, recent_list) #계정바꿔서 재시도
        ##########################################################################
        resource = resp.json()
        
        
        for i in resource['sections']: #0~8
            for k in i['layout_content']['medias']: #0~2
                if (k['media']['caption'] != None):
                    if 'text' in k['media']['caption'].keys():
                        recent_list[data['page']].append(emoji.demojize(k['media']['caption']['text'])) # 이모티콘 제거 ###############
                        print(k['media']['caption']['created_at_utc']) # 포스트의 현재 시간 1669098438 ##############################
            
        if resource['more_available']:
            data['max_id'] = resource['next_max_id']
            data['page'] = resource['next_page']
    
        else:
            recent_info['isnext']=False
    
    print('최근 게시물 수집 완료!\n')
    print('총 소요시간 : ', datetime.now() - before,'\n\n')
    return (True, recent_list)

In [6]:
def print_text(text_list):
    for i in range(len(text_list)):
        print('\n\n\n\n\n\n################################{}페이지##########################################################################\n\n\n'.format(i+1))
        for k in text_list[i]:
            for j in k.splitlines():
                if len(j)>0:
                    print(j)

In [7]:
#MAIN---------------------------------------------------------------
keyword = '수능'
# 초기값 선언
login_success = False
total_success = False
acc_list = [('seungirumd+1@gmail.com', 'pinstaw24'),('kj10522002@korea.ac.kr','dev123dev!'),('quickpass8@gmail.com','thskrl1!'),('hotkey2','gktzl2'),
           ('lshyun0510.11@gmail.com', '*thvmxmdnpdj')] #lshyun0510.dev@gmail.com / *thvmxmdnpdj (성현님 예비 id/pw)
cur_idx = -1 #현재 사용하고 있는 아이디 인덱스
max_page = 2
top_list, recent_list = list(), list()
#초기값 설정 끝

In [8]:
'''
수집 작업 시작 직전 타임 스탬프 기록
'''
print(time.strftime('%Y-%m-%d'))  # 현재시각 기준'2022-11-22' 출력 ####################################################

2022-11-22


In [9]:
while not total_success: #크롤링 성공까지 돌아가면서 시도
    cur_idx = (cur_idx+1)%len(acc_list) #원형 순환
    before = datetime.now()
    while not login_success: #로그인 성공까지 돌아가면서 시도
        print('Currently trying with ... :', acc_list[cur_idx][0]) #현재 로그인 시도 중인 계정
        session = Session()
        headers = dict()
        headers['user-agent'] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.24'
        headers['x-ig-app-id'] = '936619743392459'
        headers, session, login_success = login(acc_list[cur_idx][0], acc_list[cur_idx][1], headers, session)
        if (not login_success):
            cur_idx = (cur_idx+1)%len(acc_list) #원형 순환
    print('로그인 소요시간 : ', datetime.now() - before, '\n')
    login_success = False
    #로그인 성공인경우// 현재 cur_idx는 로그인 진입 성공한 계정 인덱스
    print('Maximum number of pages to crawl : ', max_page, '\n\n')
    
             
    total_success, top_list = hot_key_instagram_top(keyword, headers, session, max_page)
    #or
    total_success, recent_list = hot_key_instagram_recent(keyword, headers, session, max_page)

total_success = False

Currently trying with ... : seungirumd+1@gmail.com
200 {'user': True, 'userId': '56435698992', 'authenticated': True, 'oneTapPrompt': True, 'status': 'ok'}

successfully logged in...
로그인 소요시간 :  0:00:02.830593 

Maximum number of pages to crawl :  2 


인기 게시물: 1페이지....
1669082436
1668676451
1669039593
1669097226
1668754324
1668671117
1669040349
1669092503
1668867914
인기 게시물: 2페이지....
1668853956
1668756918
1668779512
1669102173
1669103125
1669020605
1668734696
1668945910
1668741614
1668693722
1669106608
1668676697
1668749952
1669100973
1668674779
1669083702
1668756985
1669080309
1668672883
1669089611
1668698103
1669022344
1668693695
1669107605
1668674307
1669106633
1669028985
1668690588
1669103009
1668642228
인기 게시물 수집 완료!

총 소요시간 :  0:00:05.640690 


최근 게시물: 1페이지....
1669107932
1669107910
1669107825
1669107797
1669107671
1669107627
1669107642
1669107605
1669107472
1669107204
1669107249
1669107074
1669107028
1669106990
1669107006
1669106819
1669106799
1669106840
1669106633
1669106688
1669

In [10]:
recent_list

[['힐링곡:melting_face:\n.\n.\n.\n.\n.\n.\n.\n#고3 #04년생 #04 #19살 #수능 #끝 #재업 #배우지망생 #감성 #sections',
  '오늘의 대련 열정지수 100000%:fire:\n(물론 항상 그랬음^^)\n\n#수험생 #수험생할인 \n친구와 함께 최대 50%할인받고 그동안 쌓인 스트레스 해소, 다이어트, 근성장 가보자고:fire: (피드 게시물 참조)\n\n:megaphone:\n유치부 초등부 중등부 고등부 성인부 입시부\n관원 상시 모집중입니다:smiling_face_with_open_hands:\n\n:martial_arts_uniform:\n1일 체험도 가능하니 연락 후\n편하게 방문해주세요:heart_suit:\n\n:telephone_receiver:\nH.P. 010-7422-6080\n\n:round_pushpin:\n대전 유성구 계룡로 150 5층\n.\n.#대전유도 #대전유도관 #대전유도장 #유성유도 #유성유도관 #봉명동유도 #만년동유도 #유성온천역유도 #유성합기도 #유성맘 #봉명맘 #judo #Dreamofjudo #수능 #경찰 #군인 #소방관 #다이어트 #바디프로필 #호신술 #운동스타그램 #만년동 #봉명동 #궁동 #충남대 #유성구 #대전맛집 #드림오브유도',
  '수험생분들 수능은 좀 씹어 드셨나요..? 저는 간신히 최저만 맞췄답니다\n\n#수능 #수능끝 #자유의몸 #냥스타그램 #냥스타 #캣 #캣스타그램 #캣스타 #새끼고양이 #아깽이 #고양이 #고양이일상 #고양이그램\xa0 #고양이집사 #초보집사 #소통 #냥이소통 #냥팔맞팔 #cat #kitty #kittylove #kittygram #catstagram #catlover  #catsofinstagram #catloversclub #catphoto #catlife #catdaily',
  ":megaphone:신라명과 수험생 격려메세지 이벤트:heart_with_ribbon::megaphone:\n\n수능이 끝난 기념으로 진행하는 신라명과 특별

In [11]:
print('인기게시물 출력 : ')
print_text(top_list)

인기게시물 출력 : 






################################1페이지##########################################################################



2023년도 대학수학능력시험(수능) 고사장에 ‘성게 머리’를 하고 나타나 주목받았던 수험생이 자신의 머리 모양에 대해 직접 언급했다.
21일 SBS ‘모닝와이드’에 출연한 ‘성게 머리’ 장기헌 씨는 “수능 날 특별하게 노리고 이 머리를 한 것이 아니고 평소에도 이 머리를 하고 다닌다”고 밝혔다.
장 씨는 “수능 볼 때, 평소 텐션이 중요하다. 평소에 그냥 이러고 다니다가 갑자기 긴장해서 머리를 다 내리고 시험 보러 가면, 긴장을 많이 할 수도 있지 않나”라며 “그래서 긴장을 해소하고자 평소 같은 마음으로 보자(라고 생각했다)”고 말했다.
‘예민한 수험생들한테는 방해될 수도 있겠다는 생각은 해보지 않았나’는 질문에 “절대적인 기준이 없지 않으냐”라며 “예를 들어 ‘두피 위, 머리가 몇㎝ 이상 솟아 있으면 부정행위로 간주된다’는 것도 없다”고 했다.
이어 “그런 걸로 치면 앞사람이 (책상) 두드리는 거나 다리 떠는 거나 헛기침하는 것도 다 잡아내야 하는데 그런 기준이 없다”며 “저로선 볼펜 딸깍거리는 사람이 더 신경쓰인다”고 덧붙였다.
그러면서도 “결실을 이루는 곳이 시험장인데, 만약 저 때문에 시험에 방해됐다고 하면 제가 큰 죄를 지은 거긴 하다”고 말하기도 했다.
앞서 수능 당일 온라인 커뮤니티에는 장 씨 목격담과 함께 그의 헤어스타일을 몰래 찍은 사진이 다수 올라온 바 있다.
:backhand_index_pointing_right:기사 전문: 프로필 링크 클릭
:camera:사진 출처: SBS '모닝와이드' 캡처, 디시인사이드
#수능 #대학수학능력시험 #머리 #헤어스타일 #수험생
수능 끝. 수고하셨습니다.
#수능 #대학수학능력시험 #수험생
사진출처:camera_with_flash:: 연합뉴스
이번주 한주도 불태워!!!:fire

In [12]:
print('최근게시물 출력 : ')
print_text(recent_list)

최근게시물 출력 : 






################################1페이지##########################################################################



힐링곡:melting_face:
.
.
.
.
.
.
.
#고3 #04년생 #04 #19살 #수능 #끝 #재업 #배우지망생 #감성 #sections
오늘의 대련 열정지수 100000%:fire:
(물론 항상 그랬음^^)
#수험생 #수험생할인 
친구와 함께 최대 50%할인받고 그동안 쌓인 스트레스 해소, 다이어트, 근성장 가보자고:fire: (피드 게시물 참조)
:megaphone:
유치부 초등부 중등부 고등부 성인부 입시부
관원 상시 모집중입니다:smiling_face_with_open_hands:
:martial_arts_uniform:
1일 체험도 가능하니 연락 후
편하게 방문해주세요:heart_suit:
:telephone_receiver:
H.P. 010-7422-6080
:round_pushpin:
대전 유성구 계룡로 150 5층
.
.#대전유도 #대전유도관 #대전유도장 #유성유도 #유성유도관 #봉명동유도 #만년동유도 #유성온천역유도 #유성합기도 #유성맘 #봉명맘 #judo #Dreamofjudo #수능 #경찰 #군인 #소방관 #다이어트 #바디프로필 #호신술 #운동스타그램 #만년동 #봉명동 #궁동 #충남대 #유성구 #대전맛집 #드림오브유도
수험생분들 수능은 좀 씹어 드셨나요..? 저는 간신히 최저만 맞췄답니다
#수능 #수능끝 #자유의몸 #냥스타그램 #냥스타 #캣 #캣스타그램 #캣스타 #새끼고양이 #아깽이 #고양이 #고양이일상 #고양이그램  #고양이집사 #초보집사 #소통 #냥이소통 #냥팔맞팔 #cat #kitty #kittylove #kittygram #catstagram #catlover  #catsofinstagram #catloversclub #catphoto #catlife #catdaily
:megaphone:신